# Iniciación de Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Sección 1: Origen de los datos** 🌐

## **1.1- Introducción**

En un proyecto determinado, la aplicación de estrategias de cómputo depende de la **información disponible del sistema (DATOS)** y de los **objetivos** específicos del estudio.

El laboratorio humedo es el primer acercamiento que tenemos a la generación de datos, mejor conocidos como: **Datos Crudos**. 


---
Datos Crudos: Registros originales tal como se recopilaron en el laboratorio sin ningún procesamiento, filtrado o interpretación.

---

Como QFB, trabajan con muestras complejas como:

> *   🩸 Sangre/plasma (biomarcadores)
> *   🔬 Tejidos (biopsias)
> *   🧫 Cultivos celulares (hepatocitos, líneas cancerosas)
> *   🦠 Aislamientos microbianos (resistencia antimicrobiana)

Como QFB, trabajan con muestras complejas (Datos Crudos):

Después de generar los datos, estos pasan a la computadora para proceder a su manejo y adaptarlas acorde a las herramientas de uso.

Las herramientas a utilizar es seleccionada basada a la pregunta del proyecto **¿Que me interesa conocer de mis datos?** 


# Beneficios:


1.   Dejar atrás los análisis unigénicos (qPCR) para adoptar enfoques multiómicos.

2.   Vincular sus muestras a mecanismos moleculares con impacto en terapias y diagnósticos


# **Sección 2: BioComputo** 🧪💻



# COLAB

## **2.1- Preprocesamiento de Datos**

### 2.1.1 scRNA-seq

### 2.1.2 RNA-seq

## 2.1.1- Visualización de Datos    

## **2.2- Herramientas Computacionales**

Son programas, plataformas o algoritmos diseñados para analizar, visualizar o interpretar datos biológicos complejos (genéticos, químicos, farmacológicos) de manera automática, rápida y precisa. 

**FUNCION:** simplificar tareas que manualmente serían imposibles o muy lentas.

**CONSEJOS:**
> Empieza con herramientas web (CIBERSORTx, STRING) antes de usar software local.

> Usa datos de ejemplo: Muchas herramientas ofrecen datasets de prueba para practicar.

> Busca tutoriales en YouTube: Hay guías visuales para cada herramienta.


**EJEMPLOS:**

> STRING (Web)
Función: Predecir interacciones entre proteínas 

> CIBERSORTx (Web)
Función: Estimar proporciones de tipos celulares en una muestra

> KEGG mapper (Web)
Función: Mapear genes o fármacos en vías bioquímicas conocidas

> hdWGCNA  (Software)
Función:  Encontrar grupos de genes que actúan juntos (módulos) en procesos biológicos.

## 2.2.1- STRING

STRING (Search Tool for Retrieval of Interacting Genes/Proteins) es una plataforma web gratuita que permite estudiar interacciones funcionales entre proteínas, genes o compuestos químicos. Combina datos experimentales, predicciones computacionales y minería de textos para construir redes biológicas.

**Enlace: https://string-db.org**

### Análisis avanzado (sin programar)
> Enriquecimiento funcional:

STRING automáticamente detecta qué vías o procesos biológicos están sobrerrepresentados en tu red (ej.: "apoptosis", "metabolismo de lípidos").

> Comparación entre especies
> Integración con fármacos:
Usa el plugin STITCH para incluir compuestos químicos en la red

## 2.2.2- Inferencia de proporción celular (CIBERSORTx) 📊

<fig>
<img    src="Edits/MTD.png"
        width=800
        height=800>
<figcaption>Fig. 1: Graphical abstract de la metodología.
</figure>

### Link a CIBERSORTx: https://cibersortx.stanford.edu/

Pasos a seguir


1.   Crear cuenta
2.   Iniciar sesíon


### Matriz de firmas genéticas

## Procesamiento de datos RNA-seq

### Librerias

In [ ]:
import pandas as pd # manipulación de datos
import matplotlib.pyplot as plt # Graficación
import time # Medición de tiempo
import numpy as np # Matematicas

### Lectura de datos

In [ ]:
singlecell=pd.read_csv("/content/drive/MyDrive/Curso de Marzo/SingleCell.csv", header=None) #Single Cell con 3,500 muestras celulares de 15 tipos celulares
singlecell=singlecell.transpose()

singlecell # Mostrar los datos en forma de tabla

### Procesamiento de datos RNA-seq

In [ ]:
# Carga de datos
rnaseq=pd.read_csv("/content/drive/MyDrive/Curso de Marzo/RNA-seq.csv") # Dirección en la que se encuentra el archivo RNA-seq
print("Base de datos \n")

rnaseq = rnaseq.rename({'gene_symbol': 'Brain Samples', 'b': 'Y'}, axis=1) # Cambiamos de nombre una columna
rnaseq.head() # Mostrar los datos en forma de tabla solo los primeros 5 elementos


### Filtrado de datos

In [ ]:
ages = ["11 yrs", "40 yrs"] #Edades en las que quieres inferir la proporción celular


# Filtara el DF
filtered_df = rnaseq[rnaseq["Age"].isin(ages)]

# Eliminar columna de edad
filtered_df=filtered_df.drop("Age", axis=1)

# Mostrar los resultados
filtered_df=filtered_df.set_index(["Brain Samples"])
print("Base de datos con Indice\n")

filtered_df.head()

# Define the Google Drive path
save_path = "/content/drive/MyDrive/Curso de Marzo/"

#Exportar los datos a un archivo tipo csv
filtered_df.to_csv(save_path+"filtered_df.csv")

### Guardar datos en formato txt

In [ ]:
# Carga de datos previamente segmentados
rnaseq=pd.read_csv(save_path+"filtered_df.csv", header=None)
print("Base de datos \n")
rnaseq=rnaseq.transpose()
# Save the file
np.savetxt(save_path+"Brain_Bulk.txt", rnaseq, fmt='%s', delimiter='\t')

### Vizialización de datos segmentados finales

In [ ]:
rnaseq=pd.read_csv(save_path+"Brain_Bulk.txt", header=None, delimiter="\t", index_col=0)
rnaseq

Hola


<fig>
<img    src="Edits/CIBERSORTx.png"
        width=800
        height=600>
<figcaption>Fig. 1: Interfaz de CIBERSORTx.
</figure>

### Matriz de firmas genéticas

### Inferencia Celular

##hdWGCNA


hdWGCNA es una herramienta bioinformática que permite analizar redes de coexpresión génica en estudios de transcriptómica que nos permite:

- Identificar grupos de genes que trabajan juntos en diferentes condiciones, como en enfermedades o en respuesta a fármacos

- Encontrar biomarcadores y posibles dianas terapéuticas analizando qué genes están más conectados en una red

- Comprender cómo un fármaco afecta la expresión génica y predecir posibles efectos adversos

- Estudiar enfermedades complejas como el cáncer o trastornos neurológicos a nivel molecular

## 2.2.3- hdWGCNA 🕸️


hdWGCNA es una herramienta bioinformática que permite analizar redes de coexpresión génica en estudios de transcriptómica que nos permite:

- Identificar grupos de genes que trabajan juntos en diferentes condiciones, como en enfermedades o en respuesta a fármacos

- Encontrar biomarcadores y posibles dianas terapéuticas analizando qué genes están más conectados en una red

- Comprender cómo un fármaco afecta la expresión génica y predecir posibles efectos adversos

- Estudiar enfermedades complejas como el cáncer o trastornos neurológicos a nivel molecular

# **Sección 3: Ciencia de Datos** 🧬💻

## 3.1- Interpretación de Resultados


La interpretación de resultados en biología computacional es el proceso de traducir datos numéricos o estadísticos en conocimiento biológicamente relevante, vinculándolos con mecanismos celulares, enfermedades o aplicaciones farmacéuticas. Se realiza en varias etapas:

> Validación técnica:

Confirmar que los resultados son robustos (ej.: repetir análisis con distintos parámetros o subconjuntos de datos).


> Contextualización biológica:

Relacionar genes, proteínas o vías identificadas con funciones conocidas usando bases de datos como KEGG, GO o Reactome.

> Integración con literatura:

Comparar resultados con estudios previos

> Herramientas de apoyo:

**Anotación funcional:** DAVID, Enrichr para identificar vías enriquecidas.

**Redes biológicas:** Cytoscape para visualizar interacciones entre genes/proteínas descubiertos.

**Plataformas farmacológicas:** DrugBank para cruzar dianas identificadas con fármacos existentes.

> Traducción a aplicaciones prácticas:

Generar hipótesis experimentales




*Siempre pregúntense: ¿Qué significa esto para la enfermedad o el fármaco que estudio?*